In [24]:
import pymc3 as pm
import pandas as pd
import numpy as np
import theano.tensor as T
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle
from theano import shared

In [65]:
df = pd.read_csv("../data/dmc2001_learn.txt", sep=";")
df = df.fillna(df.mean(numeric_only=True))
# Preprocessing: let's assume all your columns are numerical for simplicity
df['WO'] = df['WO'].replace({'W': 1, 'O': 0, 'F': 2})
train, test = train_test_split(df, test_size=0.2)

In [66]:
train.shape, test.shape

((8000, 35), (2000, 35))

In [68]:
train.head()

,ID,jahrstart,AKTIV,WO,Regiotyp,Kaufkraft,Bebautyp,Strtyp,Bonitaet,Famgr,...,Typ7,Typ8,Typ9,Abogr,PHARM1,PHARM2,PHARM3,PHARM4,PHARM5,PHARM6
4383,390277,1997,1,0,13.000000,-5.000000,2.000000,3.000000,7.00000,4.00000,...,4.000000,5.000000,4.000000,4.52335,1.000000,3.000000,4.000000,2.000000,4.000000,6.000000
4475,93207,1990,0,0,12.000000,-4.000000,1.000000,1.000000,7.00000,6.00000,...,3.860556,3.751168,3.935316,4.52335,3.742703,4.093642,4.256055,4.270869,3.858023,3.690486
3207,56049,1992,0,1,15.000000,2.000000,1.000000,1.000000,3.00000,6.00000,...,5.000000,1.000000,3.000000,2.00000,6.000000,7.000000,2.000000,6.000000,2.000000,6.000000
2757,341018,1996,1,1,11.000000,6.000000,2.000000,3.000000,7.00000,9.00000,...,4.000000,3.000000,5.000000,6.00000,6.000000,5.000000,3.000000,1.000000,6.000000,1.000000
7137,363931,1996,1,2,13.591884,2.942619,1.735179,1.619571,4.97971,5.41583,...,3.860556,3.751168,3.935316,4.52335,3.742703,4.093642,4.256055,4.270869,3.858023,3.690486


In [70]:
import theano

def create_shared_data(data):
    shared_data = {}
    for column in data.columns:
        shared_data[column] = theano.shared(data[column].values)
    return shared_data

# Convert your DataFrame to shared variables
shared_train_data = create_shared_data(train.drop('AKTIV', axis=1))
shared_test_data = create_shared_data(test.drop('AKTIV', axis=1))

with pm.Model() as model:
    # Priors for each feature
    priors = {col: pm.Normal(col, mu=0, sd=1) for col in train.columns if col != "AKTIV"}

    # Expected value using logistic function
    mu = pm.math.sigmoid(sum([priors[col] * shared_train_data[col] for col in train.columns if col != "AKTIV"]))

    # Likelihood
    AKTIV = pm.Bernoulli('AKTIV', p=mu, observed=train['AKTIV'])

    # Sample
    trace = pm.sample(20, tune=1000)

# Switch to test data and generate posterior predictive samples
for column in test.columns:
    if column in shared_test_data:
        shared_train_data[column].set_value(test[column].values)

with model:
    ppc = pm.sample_posterior_predictive(trace, samples=500)

/tmp/ipykernel_110692/1437999406.py:24: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(20, tune=1000)
Only 20 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [PHARM6, PHARM5, PHARM4, PHARM3, PHARM2, PHARM1, Abogr, Typ9, Typ8, Typ7, Typ6, Typ5, Typ4, Typ3, Typ2, Typ1, PKW_Gel, PKW_GW, PKW_KB, PKW_Lei, PKW_Di, AnzGew, AnzHH, AntDt, Altersgr, Famgr, Bonitaet, Strtyp, Bebautyp, Kaufkraft, Regiotyp, WO, jahrstart, ID]


Sampling 4 chains for 1_000 tune and 20 draw iterations (4_000 + 80 draws total) took 449 seconds.
/home/programmer/miniconda3/envs/dataProject/lib/python3.11/site-packages/pymc3/sampling.py:644: UserWarning: The number of samples is too small to check convergence reliably.
  warnings.warn("The number of samples is too small to check convergence reliably.")


In [71]:
ppc["AKTIV"].shape

(500, 2000)

In [83]:
ppc["AKTIV"][:,0].mean()

0.438

In [85]:
prediction =  np.mean(ppc["AKTIV"], axis=0)
prediction

(2000,)

In [92]:
pred = [1 if x > 0.5 else 0 for x in prediction] 
lables_test = test["AKTIV"].to_list()

In [95]:
correct_predictions = sum(p == l for p, l in zip(pred, lables_test))
accuracy = correct_predictions / len(lables_test) * 100

In [96]:
accuracy

61.650000000000006

In [102]:
def cost_matrix(prediction, labels):
    true_positive = 1.100
    false_positive = -265
    true_negative = -25
    false_negative = 662
    total_cost = 0
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for pred, label in zip(prediction, labels):
        if pred == 1 and label == 1:
            total_cost += true_positive
            true_positive += 1
        elif pred == 1 and label == 0:
            total_cost += false_positive
            false_positive += 1
        elif pred == 0 and label == 0:
            total_cost += true_negative
            true_negative += 1
        elif pred == 0 and label == 1:
            total_cost += false_negative
            false_negative += 1
        else:
            raise ValueError(f"pred {pred} label {label}")
    
    print(f"true_positive {true_positive} false_positive {false_positive} true_negative {true_negative} false_negative {false_negative}")
    return total_cost, true_positive, false_positive, true_negative, false_negative
    
    
    

In [103]:
cost_matrix(pred, lables_test)

true_positive 725 false_positive 479 true_negative 508 false_negative 288


(547037, 725, 479, 508, 288)